In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("TakeHomeExamRawLayer") \
    .getOrCreate()

25/05/22 16:41:16 WARN Utils: Your hostname, xander-xps159500 resolves to a loopback address: 127.0.1.1; using 192.168.50.20 instead (on interface wlp108s0)
25/05/22 16:41:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).

25/05/22 16:41:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.UnsupportedOperationException: getSubject is not supported
	at java.base/javax.security.auth.Subject.getSubject(Subject.java:277)
	at org.apache.hadoop.security.UserGroupInformation.getCurrentUser(UserGroupInformation.java:577)
	at org.apache.spark.util.Utils$.$anonfun$getCurrentUserName$1(Utils.scala:2416)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.util.Utils$.getCurrentUserName(Utils.scala:2416)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:329)
	at org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
	at java.base/jdk.internal.reflect.DirectConstructorHandleAccessor.newInstance(DirectConstructorHandleAccessor.java:62)
	at java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:499)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:483)
	at py4j.reflection.MethodInvoker.in

In [ ]:
df_orders = spark.read.csv("./source_data/orders.csv", header=True, inferSchema=True)
df_customers = spark.read.csv("./source_data/customers.csv", header=True, inferSchema=True)
df_products = spark.read.csv("./source_data/products.csv", header=True, inferSchema=True)
df_geography = spark.read.csv("./source_data/geography.csv", header=True, inferSchema=True)
df_order_status = spark.read.csv("./source_data/order_status.csv", header=True, inferSchema=True)


In [ ]:

# Orders voorbeeld
df_orders.printSchema()
df_orders.show(5)
print(f"Aantal rijen: {df_orders.count()}")


In [ ]:

df_orders.write.mode("overwrite").parquet("/Raw/orders")
df_customers.write.mode("overwrite").parquet("/Raw/customers")
df_products.write.mode("overwrite").parquet("/Raw/products")
df_geography.write.mode("overwrite").parquet("/Raw/geography")
df_order_status.write.mode("overwrite").parquet("/Raw/order_status")

In [ ]:

df_check = spark.read.parquet("/Raw/orders")
df_check.show